In [13]:
#Copyright 2019 Vraj Shah, Arun Kumar
#
#Licensed under the Apache License, Version 2.0 (the "License");
#you may not use this file except in compliance with the License.
#You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
#Unless required by applicable law or agreed to in writing, software
#distributed under the License is distributed on an "AS IS" BASIS,
#WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#See the License for the specific language governing permissions and
#limitations under the License.

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Model
from keras.layers import Dense, Embedding, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPool1D, Dropout, concatenate
from keras.preprocessing import text as keras_text, sequence as keras_seq
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import accuracy_score, confusion_matrix
np.random.seed(512)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from keras.layers import LeakyReLU
from keras import initializers
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.models import load_model
from keras.utils import to_categorical
import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [14]:
# define network parameters
max_features = 128
maxlen = 128

In [20]:
#read csv
dict_label = {'Usable directly numeric':0, 'Usable with extraction':1, 'Usable with Extration': 1, 'Usable with extraction ':1, 'Usable directly categorical':2, 'Unusable':3, 'Context_specific':4, 'Usable directly categorical ':2}
data = pd.read_csv('data_for_ML_num.csv')
data['y_act'] = [dict_label[i] for i in data['y_act']]
y = data.loc[:,['y_act']]
data_LSTM = pd.concat([data['Attribute_name'], data['sample_1'], data['sample_2'], data['sample_3'], data['sample_4'], data['sample_5']], axis =1)

# X_train, X_test,y_train,y_test = train_test_split(data,y, test_size=0.2)

In [21]:
data['Num of nans'] = [data['Num of nans'][i]*100/data['Total_val'][i] for i in data.index]
data['num of dist_val'] = [data['num of dist_val'][i]*100/data['Total_val'][i] for i in data.index]

data1 = data[['Num of nans', 'max_val', 'mean', 'min_val', 'num of dist_val','std_dev','castability','extractability', 'len_val']]
data1 = data1.fillna(0)

# data = data.rename(columns={'mean': 'scaled_mean', 'min_val': 'scaled_min_val', 'max_val': 'scaled_max_val','std_dev': 'scaled_std_dev','len_val': 'scaled_len_val'})
# column_names_to_normalize = ['scaled_max_val', 'scaled_mean', 'scaled_min_val','scaled_std_dev', 'scaled_len_val']
# x = data[column_names_to_normalize].values
# x = np.nan_to_num(x)
# x_scaled = StandardScaler().fit_transform(x)
# df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = data.index)
# data[column_names_to_normalize] = df_temp

# X_train, X_test,y_train,y_test = train_test_split(data,y, test_size=0.2)

# new_data = data[['scaled_min_val']]
# new_data.scaled_min_val = new_data.scaled_min_val.astype(float)
# normalized_df=(new_data-new_data.mean())/new_data.std()
# new_data = new_data[np.abs(new_data.scaled_min_val-new_data.scaled_min_val.mean()) <= (3*new_data.scaled_min_val.std())]

# for index, row in new_data.iterrows():
#     if row['scaled_min_val'] > 100000:
#         new_data.iloc[index]['scaled_min_val'] = 100000
#     if row['scaled_min_val'] < -100000:
#         new_data.iloc[index]['scaled_min_val'] = -100000        
#     print(row['scaled_min_val']) 

# q = new_data['scaled_min_val'].quantile(0.99)
# new_data = new_data[new_data['scaled_min_val'] < q]
# print(new_data)

# print(new_data.mean())
# print(new_data.std())


# scaled_features = StandardScaler().fit_transform(new_data.values)
# df = pd.DataFrame(scaled_features)
# scaled_features = MinMaxScaler().fit_transform(new_data.values)
# df = pd.DataFrame(scaled_features)

# std_scale = preprocessing.StandardScaler().fit(data[['max_val', 'min_val']])
# data = std_scale.transform(data[['max_val', 'min_val']])
# data = pd.DataFrame(data)

In [22]:
# data1.to_csv('before.csv')
# f = open('current.txt','w')
d = enchant.Dict("en_US")

for i in data.index:
    ival = data.at[i,'Attribute_name']
    if ival != 'id' and d.check(ival):
#         print >> f,ival
#         print >> f,y.at[i,'y_act']
        data1.at[i,'dictionary_item'] = 1
    else:
        data1.at[i,'dictionary_item'] = 0

# data1.to_csv('after.csv')
# f.close()

In [23]:
# print(data1.columns)

arr = data['Attribute_name'].values

vectorizer = CountVectorizer(ngram_range=(3,3),analyzer='char')
X = vectorizer.fit_transform(arr)
print(len(vectorizer.get_feature_names()))
# print(X.toarray())

# data1.to_csv('before.csv')

tempdf = pd.DataFrame(X.toarray())

data2 = pd.concat([data1,tempdf], axis=1, sort=False)

# data2.to_csv('after.csv')


6360


In [27]:
X_train, X_test,y_train,y_test = train_test_split(data2,y, test_size=0.2,random_state=100)

import pickle
filename = 'finalized_model.sav'

loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.8935710426796326
